<a href="https://colab.research.google.com/github/ChaesongYun/YT_Title_Predictor/blob/main/Youtube_Title_Predictor_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 모델 Fine Tuning

클리닝이 끝난 데이터를 BERT 자연어처리 인공지능 모델을 이용해서 fine tuning 작업을 해준다.<br>
구글에서 pre-tranined된 BERT 모델을 받아준 뒤 데이터를 맞춤 제작을 한다.
<br>
맞춤제작이 끝나면 그 모델 유튜브 제목 퀄리티를 예측할 수 있다!(●'◡'●)

1. Pre-tranined tokenizer 다운 받기
2. Data sampling. 데이터 섞기
3. 데이터셋을 training용과 validation용으로 나누기
4. 인풋 데이터 만들기
5. 학습모델 모델링
6. 모델 compile
7. 모델 fit
8. 모델 평가하기
9. 모델 다운로드

In [29]:
# 필요한 라이브러리 가져오기
!pip install transformers # Hugging Face에서 가져온다
import transformers
from transformers import BertTokenizer

# 선행된 Tokenizer 다운받기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

import tensorflow as tf
import numpy as np
import pandas as pd

In [40]:
# cleaned 데이터 가져오기
dataset = pd.read_csv("/content/drive/MyDrive/Youtube_Title_Predictor/cleaned_data.csv", index_col=False)

# 데이터 섞기
dataset = dataset.sample(frac=1)


In [50]:
# dataset을 학습용과 점검용으로 나누기
ten_perc = int(len(dataset) * 0.1)
train_titles = np.array(dataset['Title'][:-ten_perc], dtype=str)
train_labels = np.array(dataset['Target'][:-ten_perc])

valid_titles = np.array(dataset['Title'][-ten_perc:], dtype=str)
valid_labels = np.array(dataset['Target'][-ten_perc:])

train_titles[:5], valid_titles[:5]

(array(['리트리버가 도끼눈을 뜨고 주인을 노려보는 귀여운 이유 ㅋㅋㅋ',
        '사건 현장 범인은 사라지고 quot 네가 그랬지 quot 허스키는 억울해 JTBC 사건반장',
        '털 안빠지는 대형견 종류 TOP 4 강아지',
        '휴가철 급증하는 유기견 보호자에게 매몰차게 버려진 개들은 어떻게 살아갈까 구조된 강아지들의 운명 왜그러냥귀엽개',
        '유기견 사건의 결말'], dtype='<U98'),
 array(['사나운 개 웃는 개 사모예드',
        '피크타임 16마리 비숑을 위한 강형욱 훈련사의 특급 솔루션 성격 별 그룹 형성 개는훌륭하다 KBS 230529 방송',
        '인형인줄 아기 강아지 입양했어요 2개월 포메라니안 ep 1',
        '늑대를 닮은 진돗개 호피 무늬 칡개 앞에 뱀이 나타났다 KBS 굿모닝 대한민국 120612 방송',
        '강아지에게 계속 뽀뽀했더니 진돗개'], dtype='<U94'))

In [51]:
# 인풋 데이터 만들기(샘플)

i = 35
tokenized_data = tokenizer.encode_plus(train_titles[i],
                                       add_special_tokens = True,
                                       max_length = 30,
                                       padding=True,
                                       truncation=True)
tokenized_data = dict(tokenized_data)
labels = np.array(train_labels[i])

print(tokenized_data)
print(f"The label is: {labels}")
print(tokenizer.decode(tokenized_data["input_ids"]))

{'input_ids': [101, 122, 27056, 124, 38631, 9246, 12692, 80607, 9708, 118797, 21789, 11287, 9854, 54305, 82034, 9708, 12092, 9380, 28911, 10739, 9524, 11664, 10112, 119229, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
The label is: 0
[CLS] 1년에 3천 마리 이상의 진돗개가 태어난다는 진도 빅데이 알고e즘 [SEP]


In [52]:
def bert_encode(titles, maximum_length):
  input_ids = []
  attention_masks = []
  # token_type_ids = []

  for title in titles:
    encoded = tokenizer.encode_plus(title,
                                    add_special_tokens = True,
                                    max_length = maximum_length,
                                    pad_to_max_length =True,
                                    truncation=True
                                    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
    # token_type_ids.append(encoded['token_type_ids'])

  return np.array(input_ids), np.array(attention_masks)

In [53]:
train_input_ids, train_attention_masks = bert_encode(train_titles, 30)
valid_input_ids, valid_attention_masks = bert_encode(valid_titles, 30)

train_input_ids.shape, train_attention_masks.shape

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


((792, 30), (792, 30))

In [54]:
def labels(data):
  labels = []

  for i in data:
    labels.append(i)

  return np.array(labels, dtype='int32')

# print(train_labels)
train_labels = labels(train_labels)
valid_labels = labels(valid_labels)


In [55]:
# 모델 만들기

from tensorflow.keras.optimizers import Adam

def create_model(bert_model):
  input_ids = tf.keras.Input(shape=(30,), dtype='int32')
  attention_masks = tf.keras.Input(shape=(30,), dtype='int32')
  # token_type_ids = tf.keras.Input(shape=(30,), dtype='int32')

  output = bert_model([input_ids, attention_masks])
  output = output[1]
  output = tf.keras.layers.Dense(32, activation='relu')(output)
  output = tf.keras.layers.Dropout(0.2)(output)
  output = tf.keras.layers.Dense(1, activation='sigmoid')(output)

  model = tf.keras.models.Model(inputs = [input_ids, attention_masks], outputs=output)
  model.compile(Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [56]:
# 선행학습된 버트 모델 받기
from transformers import TFBertModel

bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [57]:
# Model Summary
model = create_model(bert_model)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 30)]                 0         []                            
                                                                                                  
 input_14 (InputLayer)       [(None, 30)]                 0         []                            
                                                                                                  
 tf_bert_model_5 (TFBertMod  TFBaseModelOutputWithPooli   1778534   ['input_13[0][0]',            
 el)                         ngAndCrossAttentions(last_   40         'input_14[0][0]']            
                             hidden_state=(None, 30, 76                                           
                             8),                                                            

In [60]:
# 모델 학습하기
history = model.fit([train_input_ids, train_attention_masks], train_labels,
                    validation_split=0.2, epochs = 3, batch_size=32)

Epoch 1/3
20/20 [==============================] - 5s 270ms/step - loss: 0.5718 - accuracy: 0.7077 - val_loss: 0.6551 - val_accuracy: 0.6478
Epoch 2/3
20/20 [==============================] - 5s 256ms/step - loss: 0.5445 - accuracy: 0.7409 - val_loss: 0.6487 - val_accuracy: 0.6478
Epoch 3/3
20/20 [==============================] - 6s 300ms/step - loss: 0.5059 - accuracy: 0.7694 - val_loss: 0.7171 - val_accuracy: 0.6667


In [62]:
# 모델 평가하기
model.evaluate([valid_input_ids, valid_attention_masks], valid_labels)

3/3 [==============================] - 0s 108ms/step - loss: 0.6494 - accuracy: 0.6667


[0.6493884921073914, 0.6666666865348816]

In [64]:
# 모델 평가하기
a = model.predict([valid_input_ids, valid_attention_masks])
a = tf.squeeze(a)

3/3 [==============================] - 0s 113ms/step


In [65]:
b = np.array(a)
b = np.rint(b).astype('int32')
print(a[30:50])
print(b[30:50])
print(valid_labels[30:50])

tf.Tensor(
[0.41292006 0.9120563  0.15600671 0.13988551 0.9508612  0.50196445
 0.18293615 0.9213992  0.76509213 0.89244586 0.91780096 0.18952268
 0.68899274 0.15414175 0.7173258  0.8037388  0.92591494 0.21547012
 0.8951298  0.46448013], shape=(20,), dtype=float32)
[0 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 0]
[0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 0 0 1]


In [66]:
valid_titles[30:50]

array(['한번에 제압하는 강형욱 ㄷㄷ 강형욱 개는훌륭하다 shorts', '아빠는 개 싫다',
       '용산에 개 풀겠다 육견협회 개 식용 금지법 반대 집회서 경찰과 대치 현장영상 KBS 2023 11 30',
       '동물극장 유튜브 오리지널 60점 만점에 58점 받은 천재견 사람과 깊은 교감을 나누는 마야네 5가족 동물극장 단짝 EP45 KBS 230107 방송',
       '요크셔테리어 장난치기 yorkie', '길에서 생고생한 강아지 영암 유기견 보호소',
       '와글와글 보더콜리와 허스키 목줄 놓으면 어떤 반응 보일까 2022 12 13 뉴스투데이 MBC',
       '뭐든 먹어야 산다 시고르자브종 반려견 시골강아지',
       'Every morning I didn 39 t know my dog 39 s behavior',
       '강아지 키우고 싶을 때 보면 치료 안 됨 새끼 진돗개', '눈 꺼풀 이 무곱댜 미용중에 조는 미니비숑',
       '웰시코기 8마리가 잠실 석촌호수에 나타나면 생기는 일 ㅋㅋㅋ', '덤벼 덤벼 진돗개서열 백안하우스풍뢰 진돗개 풍산개',
       '돈가스 먹으러 가는 줄 알았는데 주사 맞은 리트리버 반응ㅋㅋㅋ Mom Dog s Behavior After Giving Birth to 8 Retriever Pups',
       '릴스 1380만 조회수 월드스타 요크셔테리어 미용', '썰매견 사모예드를 눈밭에 풀어주었습니다',
       '비숑 산책 후 비숑타임',
       '시츄가 많으면 할 수 있는 것 1 The Shih Tzus 39 Tide Is Flowing What You Can Do With Lots Of Shih Tzus EP1',
       '보더콜리가 새끼양을 몰자 어미양이', '웰시코기 꼬리의 슬픈 비밀'], dtype='<U94')

In [68]:
# Model Saving
model.save("bert_model_v1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [69]:
# 모델 다운로드 받기
from google.colab import files

files.download('bert_model_v1.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>